In [519]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestRegressor

## 1. Import Data:

## 2. Defining Functions:

### 2.1 Preprocessing Functions:

In [402]:
def calc_change_sentiment(data, col):
    change_in_sent = []
    change_in_sent.append(data[col][0])
    for i in range(1,len(data[col])):
        if data[col][i] == 0:
            change_in_sent.append(0)
        elif data[col][i] < 0 or data[col][i] > 0:
            dif = data[col][i] - data[col][(i-1)]
            change_in_sent.append(dif)
    return change_in_sent

def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)       
    return input_txt
    
def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    tweets = np.vectorize(remove_pattern)(tweets, "b'")
    tweets = np.vectorize(remove_pattern)(tweets, 'b"')


    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [403]:
def classify_news(dataframe, datetime_column_name):

    day22, day23, day24, day25, day26, day27, day28, day29, day30, day31, day32, day33, day34 = [],[],[],[],[],[],[],[],[],[],[],[],[]

    for i in range(len(dataframe[datetime_column_name])):
        #if dataframe[datetime_column_name][i].day == 21 and dataframe[datetime_column_name][i].hour > 17:day22.append(i)
        if dataframe[datetime_column_name][i].day == 22 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day22.append(i)
        #if dataframe[datetime_column_name][i].day == 22 and dataframe[datetime_column_name][i].hour > 17:day23.append(i)
        elif dataframe[datetime_column_name][i].day == 23 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day23.append(i)
        #if dataframe[datetime_column_name][i].day == 23 and dataframe[datetime_column_name][i].hour > 17:day24.append(i)
        elif dataframe[datetime_column_name][i].day == 24 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day24.append(i)       
        #if dataframe[datetime_column_name][i].day == 24 and dataframe[datetime_column_name][i].hour > 17:day25.append(i)
        elif dataframe[datetime_column_name][i].day == 25 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day25.append(i)
        #if dataframe[datetime_column_name][i].day == 25 and dataframe[datetime_column_name][i].hour > 17:day26.append(i)
        elif dataframe[datetime_column_name][i].day == 26 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day26.append(i)
        #if dataframe[datetime_column_name][i].day == 26 and dataframe[datetime_column_name][i].hour > 17:day27.append(i)
        elif dataframe[datetime_column_name][i].day == 27 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day27.append(i)
        #if dataframe[datetime_column_name][i].day == 27 and dataframe[datetime_column_name][i].hour > 17:day28.append(i)
        elif dataframe[datetime_column_name][i].day == 28 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day28.append(i)
        #if dataframe[datetime_column_name][i].day == 28 and dataframe[datetime_column_name][i].hour > 17:day29.append(i)
        elif dataframe[datetime_column_name][i].day == 29 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day29.append(i)
        #if dataframe[datetime_column_name][i].day == 29 and dataframe[datetime_column_name][i].hour > 17:day30.append(i)
        elif dataframe[datetime_column_name][i].day == 30 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day30.append(i)
        elif dataframe[datetime_column_name][i].day == 1 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day31.append(i)
        elif dataframe[datetime_column_name][i].day == 2 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day32.append(i)
        elif dataframe[datetime_column_name][i].day == 3 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day33.append(i)
        elif dataframe[datetime_column_name][i].day == 4 and (dataframe[datetime_column_name][i].hour <= 15 and dataframe[datetime_column_name][i].hour >= 9):
            day34.append(i)

        else:
            pass
    news_d22, news_d23,news_d24,news_d25,news_d26,news_d27,news_d28,news_d29,news_d30,news_d31,news_d32,news_d33,news_d34 = dataframe.iloc[day22],dataframe.iloc[day23],dataframe.iloc[day24],dataframe.iloc[day25], dataframe.iloc[day26], dataframe.iloc[day27],dataframe.iloc[day28],dataframe.iloc[day29],dataframe.iloc[day30],dataframe.iloc[day31], dataframe.iloc[day32],dataframe.iloc[day33],dataframe.iloc[day34]
    return news_d22, news_d23,news_d24,news_d25,news_d26,news_d27,news_d28,news_d29,news_d30,news_d31,news_d32,news_d33,news_d34


In [404]:
def preprocess_headlines(data):
    data.drop_duplicates(subset='headline',keep=False, inplace=True)
    data.drop('ticker', axis=1, inplace=True)
    data.set_index('date_time', inplace=True)
    data_30m = data.resample('30min').median().ffill().reset_index()
    change_in_sent=calc_change_sentiment(data_30m, 'compound')
    data_30m['change in sentiment headlines'] = change_in_sent

    news_d22, news_d23,news_d24,news_d25,news_d26,news_d27,news_d28,news_d29,news_d30,news_d31,news_d32,news_d33,news_d34 = classify_news(data_30m, 'date_time')

    news_d23_red,news_d24_red,news_d25_red,news_d26_red,news_d27_red,news_d28_red,news_d29_red,news_d30_red,news_d31_red,news_d32_red,news_d33_red,news_d34_red = news_d23.iloc[4:],news_d24.iloc[1:],news_d25.iloc[1:],news_d26.iloc[1:],news_d27.iloc[1:],news_d28.iloc[1:],news_d29.iloc[1:],news_d30.iloc[1:],news_d31.iloc[1:],news_d32.iloc[1:],news_d33.iloc[1:],news_d34.iloc[1:]

    frames_news = [news_d23_red,news_d24_red, news_d25_red, news_d28_red]
    netflix_headlines_30m_d23_24_25 = pd.concat(frames_news)
    return netflix_headlines_30m_d23_24_25
    

In [405]:
def preprocess_posts(dataframe):
    vader = SentimentIntensityAnalyzer()
    dataframe['tweet_text'] = clean_tweets(dataframe['tweet_text'])
    scores = dataframe['tweet_text'].apply(vader.polarity_scores).tolist()
    scores_df = pd.DataFrame(scores)

    df = dataframe.join(scores_df, rsuffix='_right')
    df = df[['timestamp','tweet_text','followers_count','neg','neu','pos','compound']]
    df['timestamp'] = df['timestamp'].dt.tz_localize('UTC').dt.tz_convert('America/Montreal').dt.tz_localize(None)
    df['scaled_followers_count'] =(df['followers_count']/df['followers_count'].max()) + 1
    df['adj compound'] = df['compound']*df['scaled_followers_count']
    df.set_index('timestamp', inplace=True)

    twitter_df_hourly = df.resample('30min').median().ffill().reset_index()
    change_in_sent = calc_change_sentiment(twitter_df_hourly, 'adj compound')
    twitter_df_hourly['change in sentiment twitter'] = change_in_sent

    tw_news_d22,tw_news_d23,tw_news_d24,tw_news_d25,tw_news_d26,tw_news_d27,tw_news_d28,tw_news_d29,tw_news_d30,tw_news_d31,tw_news_d32,tw_news_d33,tw_news_d34 = classify_news(twitter_df_hourly, 'timestamp')

    tw_news_d23_30m,tw_news_d24_30m,tw_news_d25_30m, tw_news_d26_30m,tw_news_d27_30m,tw_news_d28_30m,tw_news_d29_30m,tw_news_d30_30m,tw_news_d31_30m,tw_news_d32_30m,tw_news_d33_30m,tw_news_d34_30m = tw_news_d23.iloc[4:],tw_news_d24.iloc[1:],tw_news_d25.iloc[1:],tw_news_d26.iloc[1:],tw_news_d27.iloc[1:],tw_news_d28.iloc[1:],tw_news_d29.iloc[1:],tw_news_d30.iloc[1:],tw_news_d31.iloc[1:],tw_news_d32.iloc[1:],tw_news_d33.iloc[1:],tw_news_d34.iloc[1:]

    frames = [tw_news_d23_30m,tw_news_d24_30m,tw_news_d25_30m, tw_news_d28_30m,tw_news_d29_30m,tw_news_d30_30m,tw_news_d31_30m,tw_news_d32_30m,tw_news_d33_30m,tw_news_d34_30m]
    tw_news_30min = pd.concat(frames)
    return tw_news_30min

### 2.2 Modeling Functions:

In [635]:
def baseline_model(data):
    data['Datetime'] = data['Datetime'].dt.tz_convert('America/Montreal').dt.tz_localize(None)
    # data_SMA = data['Adj Close'].rolling(window=3).mean().shift(1)
    # data['SMA(3)'] = data_SMA
    pred = data['SMA(3)'][3:]
    actu = data['Adj Close'][3:]
    rmse = np.sqrt(mean_squared_error(actu,pred))
    r2_sco = r2_score(actu,pred)
    # print('Root Mean Squared Error: ',rmse)
    # print('R2 Score: ', r2_sco)
    return rmse, r2_sco

In [653]:
def linear_modeling_no_sentiment(dataframe):
    i = len(dataframe['t+1'])-4
    y_train, y_test = dataframe['t+1'][3:i], dataframe['t+1'][i:-1]
    X_train, X_test = dataframe[['Adj Close','Scaled Volume','SMA(3)']][3:i], dataframe[['Adj Close','Scaled Volume','SMA(3)']][i:-1]

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    
    rmse = np.sqrt(mean_squared_error(y_test,predictions))
    r2_sco = r2_score(y_test,predictions)
    # print('LR Root Mean Squared Error: ',rmse)
    # print('LR R2 Score: ', r2_sco, '\n')
    
    reg = SGDRegressor(random_state=42)
    reg.fit(X_train, y_train)
    predictions2 = reg.predict(X_test)
    rmse2 = np.sqrt(mean_squared_error(y_test,predictions2))
    r2_sco2 = r2_score(y_test,predictions2)
    # print('SGD Root Mean Squared Error: ',rmse2)
    # print('SGD R2 Score: ', r2_sco2)
    return rmse,r2_sco,rmse2,r2_sco2

In [637]:
def linear_modeling_headlines(dataframe):
    i = len(dataframe['t+1'])-4
    y_train, y_test = dataframe['t+1'][:i], dataframe['t+1'][i:-1]
    X_train, X_test = dataframe[['Adj Close','Scaled Volume','compound','SMA(3)']][:i], dataframe[['Adj Close','Scaled Volume','compound','SMA(3)']][i:-1]

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,predictions))
    r2_sco = r2_score(y_test,predictions)
    # print('LR Root Mean Squared Error: ',rmse)
    # print('LR R2 Score: ', r2_sco,'\n')
    
    reg = SGDRegressor(random_state=42)
    reg.fit(X_train, y_train)
    predictions2 = reg.predict(X_test)
    rmse2 = np.sqrt(mean_squared_error(y_test,predictions2))
    r2_sco2 = r2_score(y_test,predictions2)
    # print('SGD Root Mean Squared Error: ',rmse2)
    # print('SGD R2 Score: ', r2_sco2)
    return rmse,r2_sco,rmse2,r2_sco2

In [638]:
def linear_model_twitter(dataframe):
    i = len(dataframe['t+1'])-4
    y_train, y_test = dataframe['t+1'][:i], dataframe['t+1'][i:-1]
    X_train, X_test = dataframe[['Adj Close','Scaled Volume','compound','SMA(3)']][:i], dataframe[['Adj Close','Scaled Volume','compound','SMA(3)']][i:-1]

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,predictions))
    r2_sco = r2_score(y_test,predictions)
    # print('LR Root Mean Squared Error: ',rmse)
    # print('LR R2 Score: ', r2_sco,'\n')

    reg = SGDRegressor(random_state=42)
    reg.fit(X_train, y_train)
    predictions2 = reg.predict(X_test)
    rmse2 = np.sqrt(mean_squared_error(y_test,predictions2))
    r2_sco2 = r2_score(y_test,predictions2)
    # print('SGD Root Mean Squared Error: ',rmse2)
    # print('SGD R2 Score: ', r2_sco2)
    return rmse,r2_sco,rmse2,r2_sco2

In [639]:
def multi_model_full(dataframe):
    i = len(dataframe['t+1'])-4
    y_train, y_test = dataframe['t+1'][:i], dataframe['t+1'][i:-1]
    X_train, X_test = dataframe[['Adj Close','Scaled Volume','compound_y','compound_x','SMA(3)']][:i], dataframe[['Adj Close','Scaled Volume','compound_y','compound_x','SMA(3)']][i:-1]

    lm = LinearRegression()
    lm.fit(X_train,y_train)
    predictions = lm.predict(X_test)
    rmse = np.sqrt(mean_squared_error(y_test,predictions))
    r2_sco = r2_score(y_test,predictions)
    #print('LR Root Mean Squared Error: ',rmse)
    #print('LR R2 Score: ', r2_sco,'\n')

    reg = SGDRegressor(random_state=42)
    reg.fit(X_train, y_train)
    predictions2 = reg.predict(X_test)
    rmse2 = np.sqrt(mean_squared_error(y_test,predictions2))
    r2_sco2 = r2_score(y_test,predictions2)
    #print('SGD Root Mean Squared Error: ',rmse2)
    #print('SGD R2 Score: ', r2_sco2,'\n')

    xg_reg = xgb.XGBRegressor(booster='gbtree', colsample_bytree = 0.7, eta = 0.03, max_depth = 3, n_estimators = 10000)
    xg_reg.fit(X_train,y_train)
    preds3 = xg_reg.predict(X_test)
    rmse3 = np.sqrt(mean_squared_error(y_test, preds3))
    r2_sco3 = r2_score(y_test,preds3)
    #print('XGB Mean Squared Error: ',rmse3)
    #print('XGB R2 Score: ', r2_sco3)

    rf_regr = RandomForestRegressor(n_estimators=1000, criterion='mse', max_depth=2, random_state=42)
    rf_regr.fit(X_train,y_train)
    preds4 = rf_regr.predict(X_test)
    rmse4 = np.sqrt(mean_squared_error(y_test, preds4))
    r2_sco4 = r2_score(y_test,preds4)
    return rmse,r2_sco,rmse2,r2_sco2,rmse3,r2_sco3,rmse4,r2_sco4

## 3. Pipeline:

In [685]:
# 1. Historical Stock Data:
stock_df = pd.read_csv('~/LighthouseLabs-Final/1. Stock_Data/TSLA_data.csv', parse_dates=['Datetime'])

# 2. Headline Data:
headlines1 = pd.read_csv('~/LighthouseLabs-Final/2. FinViz_Headline_Data/TSLA_data_2020-09-25-10-14.csv', index_col=1, parse_dates=[['date','time']])
headlines2 = pd.read_csv('~/LighthouseLabs-Final/2. FinViz_Headline_Data/TSLA_data_2020-09-28-22-15.csv', index_col=1, parse_dates=[['date','time']])
frames = [headlines1, headlines2]
headlines_df = pd.concat(frames)
headlines_df.drop_duplicates(subset='headline',keep='first',inplace=True)

# 3. Twitter Data:
twitter_d23 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/TSLA_2020-09-23.csv', parse_dates=['timestamp'])
twitter_d23_1 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/TSLA_2020-09-23-21-07.csv', parse_dates=['timestamp'])
twitter_d24 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/TSLA_2020-09-24.csv', parse_dates=['timestamp'])
#twitter_d24_1 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/FB_2020-09-24-13-22.csv', parse_dates=['timestamp'])
twitter_d25 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/TSLA_2020-09-25.csv', parse_dates=['timestamp'])
twitter_d28 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/NVDA_2020-09-28.csv', parse_dates=['timestamp'])
twitter_d28_2 = pd.read_csv('~/LighthouseLabs-Final/3. Twitter_Data/TSLA_2020-09-28_3.csv', parse_dates=['timestamp'])
frames = [twitter_d23,twitter_d23_1, twitter_d24, twitter_d25, twitter_d28,twitter_d28_2]
twitter_df = pd.concat(frames)
twitter_df.drop_duplicates(subset='tweet_text',keep='first', inplace=True)

In [686]:
def evaluate_models(baseline_df, headline_df, twitter_df):
    #1. Baseline:
    baseline_rmse, baseline_r2 = baseline_model(baseline_df)
    baseline_df2 = baseline_df
    baseline_df2['t+1'] = baseline_df2['Adj Close'].shift(-1)
    lm_baseline_rmse, lm_baseline_r2, sgd_baseline_rmse, sgd_baseline_r2 = linear_modeling_no_sentiment(baseline_df2)
    #2. Headline Final Merge:
    headlines_final = preprocess_headlines(headline_df)
    with_headlines_df = stock_df.merge(headlines_final, left_on='Datetime', right_on='date_time').drop('date_time',axis=1)
    with_headlines_df['t+1'] = with_headlines_df['Adj Close'].shift(-1)
    #3. Twitter Final Merge:
    final_twitter = preprocess_posts(twitter_df)
    with_twitter_df = stock_df.merge(final_twitter, left_on='Datetime', right_on='timestamp').drop('timestamp',axis=1)
    with_twitter_df['t+1'] = with_twitter_df['Adj Close'].shift(-1)
    #4. Full Merge:
    full_df = with_twitter_df.merge(headlines_final, left_on='Datetime', right_on='date_time').drop('date_time',axis=1)
    full_df['t+1'] = full_df['Adj Close'].shift(-1)
    #5. Evaluating Models:
    lm_headlines_rmse, lm_headlines_r2, sgd_headlines_rmse, sgd_headlines_r2 = linear_modeling_headlines(with_headlines_df)
    lm_twitter_rmse, lm_twitter_r2, sgd_twitter_rmse, sgd_twitter_r2 = linear_model_twitter(with_twitter_df)
    lm_all_rmse, lm_all_r2, sgd_all_rmse, sgd_all_r2, xgb_all_rmse, xgb_all_r2, rf_all_rmse, rf_all_r2 = multi_model_full(full_df)
    #6. Store in dict:
    result_dict = {
    'RMSE - Baseline':baseline_rmse, 'R2 - Baseline':baseline_r2, 'Linear RMSE - Baseline':lm_baseline_rmse, 'Linear R2 - Baseline':lm_baseline_r2, 'SGD RMSE - Baseline':sgd_baseline_rmse, 'SGD R2 - Baseline':sgd_baseline_r2,
    'Linear RMSE - Only Headlines': lm_headlines_rmse, 'Linear R2 - Only Headlines':lm_headlines_r2, 'SGD RMSE - Only Headlines':sgd_headlines_rmse, 'SGD R2 - Only Headlines':sgd_headlines_r2,
    'Linear RMSE - Only Twitter':lm_twitter_rmse, 'Linear R2 - Only Twitter':lm_twitter_r2, 'SGD RMSE - Only Twitter':sgd_twitter_rmse, 'SGD R2 - Only Twitter':sgd_twitter_r2,
    'Linear RMSE - All':lm_all_rmse, 'Linear R2 - All':lm_all_r2, 'SGD RMSE - All':sgd_all_rmse, 'SGD R2 - All':sgd_all_r2, 'XGB RMSE - All':xgb_all_rmse, 'XGB R2 - All':xgb_all_r2, 'RF RMSE - All':rf_all_rmse,'RF R2 - All':rf_all_r2}
    #7. Convert to DataFrame:
    result_df = pd.DataFrame.from_dict(result_dict, orient='index', columns=['Values'])
    #result_df.to_csv('~/LighthouseLabs-Final/Report_Analysis/AAPL_complete_analysis.csv')
    return result_df, full_df

In [687]:
result_df, full_df = evaluate_models(stock_df, headlines_df, twitter_df)
result_df

,Values
RMSE - Baseline,6.213041e+00
R2 - Baseline,8.142633e-01
Linear RMSE - Baseline,2.080228e+00
Linear R2 - Baseline,-5.703734e+01
SGD RMSE - Baseline,9.298627e+13
SGD R2 - Baseline,-1.159641e+29
Linear RMSE - Only Headlines,1.999692e+00
Linear R2 - Only Headlines,-5.263052e+01
SGD RMSE - Only Headlines,9.194230e+13
SGD R2 - Only Headlines,-1.133748e+29


In [688]:
result_df.to_csv('~/LighthouseLabs-Final/Report_Analysis/TSLA_complete_analysis.csv')
print('Saved!')

Saved!
